In [41]:
def get_word_dict(sentences):
    # create vocab of words
    word_dict = {}
    for sent in sentences:
        for word in sent.split():
            if word not in word_dict:
                word_dict[word] = ''
    word_dict['<s>'] = ''
    word_dict['</s>'] = ''
    word_dict['<p>'] = ''
    return word_dict

dict = get_word_dict(["the brown fox jumped over the lazy dog","the tree is green"])
print(len(dict))
print(dict.keys())
print(dict)
print("==================")



13
dict_keys(['the', 'brown', 'fox', 'jumped', 'over', 'lazy', 'dog', 'tree', 'is', 'green', '<s>', '</s>', '<p>'])
{'the': '', 'brown': '', 'fox': '', 'jumped': '', 'over': '', 'lazy': '', 'dog': '', 'tree': '', 'is': '', 'green': '', '<s>': '', '</s>': '', '<p>': ''}


In [48]:
def get_glove(word_dict, glove_path):
    # create word_vec with glove vectors
    word_vec = {}
    with open(glove_path) as f:
        for line in f:
            word, vec = line.split(' ', 1)
            if word in word_dict:
                word_vec[word] = np.array(list(map(float, vec.split())))
    print('Found {0}(/{1}) words with glove vectors'.format(
                len(word_vec), len(word_dict)))
    return word_vec
word_vec = get_glove(dict,"/home/dc/InferSent/dataset/GloVe/glove.840B.300d.txt")
print(word_vec.keys())
print(type(word_vec["the"]))
print(word_vec["the"].shape)
print()

def build_vocab(sentences, glove_path):
    '''
    input: sentences. list of words
    output: dict of words->embeddings
    '''
    word_dict = get_word_dict(sentences)
    word_vec = get_glove(word_dict, glove_path)
    print('Vocab size : {0}'.format(len(word_vec)))
    return word_vec
#dict of word/embeddings
word_vec = build_vocab(["the orange hair man has a big mouth"],"/home/dc/InferSent/dataset/GloVe/glove.840B.300d.txt")
print(word_vec['the'].shape)
print(word_vec['man'].shape)
print(word_vec['orange'].shape)


Found 13(/13) words with glove vectors
dict_keys(['the', 'is', 'over', 'green', 'dog', 'tree', 'brown', 'lazy', 'jumped', 'fox', '<p>', '</s>', '<s>'])
<class 'numpy.ndarray'>
(300,)

Found 11(/11) words with glove vectors
Vocab size : 11
(300,)
(300,)
(300,)


In [51]:
from data import get_nli, get_batch, build_vocab
import numpy as np
import argparse
import torch
import os
import sys
import time

#valid=dev
train, valid, test = get_nli('/home/dc/InferSent/dataset/SNLI')
print(type(train),type(valid),type(test))

parser = argparse.ArgumentParser(description='NLI training')
# paths
parser.add_argument("--nlipath", type=str, default='/home/dc/InferSent/dataset/SNLI', help="NLI data path (SNLI or MultiNLI)")
parser.add_argument("--outputdir", type=str, default='savedir/', help="Output directory")
parser.add_argument("--outputmodelname", type=str, default='model.pickle')


# training
parser.add_argument("--n_epochs", type=int, default=20)
parser.add_argument("--batch_size", type=int, default=64)
parser.add_argument("--dpout_model", type=float, default=0., help="encoder dropout")
parser.add_argument("--dpout_fc", type=float, default=0., help="classifier dropout")
parser.add_argument("--nonlinear_fc", type=float, default=0, help="use nonlinearity in fc")
parser.add_argument("--optimizer", type=str, default="sgd,lr=0.1", help="adam or sgd,lr=0.1")
parser.add_argument("--lrshrink", type=float, default=5, help="shrink factor for sgd")
parser.add_argument("--decay", type=float, default=0.99, help="lr decay")
parser.add_argument("--minlr", type=float, default=1e-5, help="minimum lr")
parser.add_argument("--max_norm", type=float, default=5., help="max norm (grad clipping)")

# model
parser.add_argument("--encoder_type", type=str, default='LSTMEncoder', help="see list of encoders")
parser.add_argument("--enc_lstm_dim", type=int, default=2048, help="encoder nhid dimension")
parser.add_argument("--n_enc_layers", type=int, default=1, help="encoder num layers")
parser.add_argument("--fc_dim", type=int, default=512, help="nhid of fc layers")
parser.add_argument("--n_classes", type=int, default=3, help="entailment/neutral/contradiction")
parser.add_argument("--pool_type", type=str, default='max', help="max or mean")

# gpu
parser.add_argument("--gpu_id", type=int, default=0, help="GPU ID")
parser.add_argument("--seed", type=int, default=1234, help="seed")


params, _ = parser.parse_known_args()

# set gpu device
torch.cuda.set_device(params.gpu_id)

# print parameters passed, and all parameters
print('\ntogrep : {0}\n'.format(sys.argv[1:]))
print(params)


print(len(train),len(valid),len(test))
print(train.keys())
train_s1,train_s2,train_labels = train['s1'],train['s2'],train['label']
print(len(train_s1),len(train_s2),len(train_labels))
W2V_PATH = "/home/dc/InferSent/dataset/GloVe/glove.840B.300d.txt"
train, valid, test = get_nli(params.nlipath)
word_vec = build_vocab(train['s1'] + train['s2'] +
                       valid['s1'] + valid['s2'] +
                       test['s1'] + test['s2'], W2V_PATH)

for split in ['s1', 's2']:
    for data_type in ['train', 'valid', 'test']:
        eval(data_type)[split] = np.array([['<s>'] +
            [word for word in sent.split() if word in word_vec] +
            ['</s>'] for sent in eval(data_type)[split]])

params.word_emb_dim = 300
#train_s1 is list
print(len(train_s1),len(train_s2),len(train_labels))
print(type(train_s1[0]))


** TRAIN DATA : Found 549367 pairs of train sentences.
** DEV DATA : Found 9842 pairs of dev sentences.
** TEST DATA : Found 9824 pairs of test sentences.
<class 'dict'> <class 'dict'> <class 'dict'>

togrep : ['-f', '/run/user/1000/jupyter/kernel-69e5a421-fb17-46e0-91fa-010a3f36109f.json']

Namespace(batch_size=64, decay=0.99, dpout_fc=0.0, dpout_model=0.0, enc_lstm_dim=2048, encoder_type='LSTMEncoder', fc_dim=512, gpu_id=0, lrshrink=5, max_norm=5.0, minlr=1e-05, n_classes=3, n_enc_layers=1, n_epochs=20, nlipath='/home/dc/InferSent/dataset/SNLI', nonlinear_fc=0, optimizer='sgd,lr=0.1', outputdir='savedir/', outputmodelname='model.pickle', pool_type='max', seed=1234)
3 3 3
dict_keys(['s1', 's2', 'label'])
549367 549367 549367
** TRAIN DATA : Found 549367 pairs of train sentences.
** DEV DATA : Found 9842 pairs of dev sentences.
** TEST DATA : Found 9824 pairs of test sentences.
Found 38957(/43479) words with glove vectors
Vocab size : 38957
549367 549367 549367
<class 'str'>


In [54]:
print(type(train['s1']))
print(type(train['s1'][0]))

print(train['s1'][0])
print(train['s2'][0])
print(train['label'][0])
#where are these turned into embeddings for input? 

s1 = train['s1']
s2 = train['s2']
target = train['label']
stidx=0

def get_batch(batch, word_vec):
    '''
    input: list of words which are sentences
    output: tensor of embeddings which are sentences
    '''
    # sent in batch in decreasing order of lengths (bsize, max_len, word_dim)
    lengths = np.array([len(x) for x in batch])
    max_len = np.max(lengths)
    embed = np.zeros((max_len, len(batch), 300))

    for i in range(len(batch)):
        for j in range(len(batch[i])):
            embed[j, i, :] = word_vec[batch[i][j]]

    return torch.from_numpy(embed).float(), lengths



s1_batch, s1_len = get_batch(s1[stidx:stidx + params.batch_size],
                                     word_vec)
s2_batch, s2_len = get_batch(s2[stidx:stidx + params.batch_size],
                                     word_vec)
print(type(s1_batch))
print(s1_batch.size())
#print(s1_batch) #tensor prints out values unless it is a single value then use tensor.item()
#convert pytorch tensor to numpy 
n = s1_batch.numpy()
print(n.shape)

<class 'numpy.ndarray'>
<class 'list'>
['<s>', 'A', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.', '</s>']
['<s>', 'A', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition', '.', '</s>']
1
<class 'torch.Tensor'>
torch.Size([29, 64, 300])
(29, 64, 300)
